# Amazon SageMaker script mode using Scikit Learn


---

<div class="alert alert-block alert-info">
⚠️ The latest SageMaker Distribution image version known to work with this notebook is <code>3.3.0</code>. If you encounter problems with other versions, please downgrade to version <code>3.3.0</code>. <b>To do so, you must stop your JupyterApp, downgrade the SageMaker Distribution image to <code>3.3.0</code> and restart the JupyterLabApp for the changes to take effect</b>.</div>

---

This lab demonstrates how to deploy a single machine learning model using Amazon SageMaker with `Scikit-Learn` in script mode. Script mode allows you to use your own training scripts with greater flexibility while leveraging SageMaker's managed training and deployment infrastructure.

With SageMaker script mode, you can:

- Write your own custom training code using familiar frameworks
- Maintain complete control over your algorithms and models
- Easily transition existing Scikit-Learn code to run on SageMaker
- Take advantage of SageMaker's optimized infrastructure for training and hosting

In this lab, we'll build and deploy a Scikit-Learn model that predicts housing prices based on various features. This straightforward use case provides an excellent foundation to understand how SageMaker script mode works with the Scikit-Learn framework.

SageMaker's Scikit-Learn container includes the framework and dependencies needed to train and serve models, while script mode gives you the flexibility to customize your training process. This approach combines the convenience of managed infrastructure with the control data scientists need over their modeling workflow.

The techniques demonstrated in this notebook can be applied to other machine learning problems where you want to leverage your own custom Scikit-Learn code while taking advantage of SageMaker's scalable training and deployment capabilities.

## Generate synthetic data for housing model

The code below contains helper functions to generate synthetic data representing the features of a house.

The first entry in the array is the randomly generated price of a house. The remaining entries are the features (i.e. number of bedroom, square feet, number of bathrooms, etc.).

These functions will be used to generate synthetic data for training, validation, and testing. It will also allow us to submit synthetic payloads for inference to test our endpoint from a model trained in script mode.

In [1]:
!pip install -U sagemaker

In [2]:
import numpy as np
import pandas as pd
import time

In [3]:
NUM_HOUSES = 1000
LOCATION = "NewYork_NY"
MAX_YEAR = 2019

In [4]:
def gen_price(house):
    _base_price = int(house["SQUARE_FEET"] * 150)
    _price = int(
        _base_price
        + (10000 * house["NUM_BEDROOMS"])
        + (15000 * house["NUM_BATHROOMS"])
        + (15000 * house["LOT_ACRES"])
        + (15000 * house["GARAGE_SPACES"])
        - (5000 * (MAX_YEAR - house["YEAR_BUILT"]))
    )
    return _price

In [5]:
def gen_random_house():
    _house = {
        "SQUARE_FEET": int(np.random.normal(3000, 750)),
        "NUM_BEDROOMS": np.random.randint(2, 7),
        "NUM_BATHROOMS": np.random.randint(2, 7) / 2,
        "LOT_ACRES": round(np.random.normal(1.0, 0.25), 2),
        "GARAGE_SPACES": np.random.randint(0, 4),
        "YEAR_BUILT": min(MAX_YEAR, int(np.random.normal(1995, 10))),
    }
    _price = gen_price(_house)
    return [
        _price,
        _house["YEAR_BUILT"],
        _house["SQUARE_FEET"],
        _house["NUM_BEDROOMS"],
        _house["NUM_BATHROOMS"],
        _house["LOT_ACRES"],
        _house["GARAGE_SPACES"],
    ]

In [6]:
def gen_houses(num_houses):
    _house_list = []
    for i in range(num_houses):
        _house_list.append(gen_random_house())
    _df = pd.DataFrame(
        _house_list,
        columns=[
            "PRICE",
            "YEAR_BUILT",
            "SQUARE_FEET",
            "NUM_BEDROOMS",
            "NUM_BATHROOMS",
            "LOT_ACRES",
            "GARAGE_SPACES",
        ],
    )
    return _df

## Train a house value prediction model

In the follow section, we are setting up the code to train a house price prediction model.

As such, we will launch a training job using the AWS Managed container for Scikit Learn via the Sagemaker SDK using the `SKLearn` estimator class.

In this notebook, we will be using the AWS Managed Scikit Learn image for both training and inference - this image provides native support for launching endpoints backed by SKLearn models.

In [7]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.inputs import TrainingInput
import boto3
from time import gmtime, strftime

s3 = boto3.resource("s3")

sagemaker_session = sagemaker.Session()
role = get_execution_role()

BUCKET = sagemaker_session.default_bucket()
TRAINING_FILE = "training.py"
INFERENCE_FILE = "inference.py"
SOURCE_DIR = "source_dir"

DATA_PREFIX = "DEMO_SCRIPT_MODE_SCIKIT_V1"
MODEL_ARTIFACTS = "model_artifacts"

TRAIN_INSTANCE_TYPE = "ml.m5.xlarge"
ENDPOINT_INSTANCE_TYPE = "ml.m5.xlarge"

CUR = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
ENDPOINT_NAME = "script-mode-sklearn-housing-V1" + "-" + CUR

MODEL_NAME = ENDPOINT_NAME

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


### Split a given dataset into train, validation, and test

The code below will generate 3 sets of data. 1 set to train, 1 set for validation and 1 for testing.

In [8]:
from sklearn.model_selection import train_test_split

SEED = 7
SPLIT_RATIOS = [0.6, 0.3, 0.1]


def split_data(df):
    # split data into train and test sets
    seed = SEED
    val_size = SPLIT_RATIOS[1]
    test_size = SPLIT_RATIOS[2]

    num_samples = df.shape[0]
    X1 = df.values[:num_samples, 1:]  # keep only the features, skip the target, all rows
    Y1 = df.values[:num_samples, :1]  # keep only the target, all rows

    # Use split ratios to divide up into train/val/test
    X_train, X_val, y_train, y_val = train_test_split(
        X1, Y1, test_size=(test_size + val_size), random_state=seed
    )
    # Of the remaining non-training samples, give proper ratio to validation and to test
    X_test, X_test, y_test, y_test = train_test_split(
        X_val, y_val, test_size=(test_size / (test_size + val_size)), random_state=seed
    )
    # reassemble the datasets with target in first column and features after that
    _train = np.concatenate([y_train, X_train], axis=1)
    _val = np.concatenate([y_val, X_val], axis=1)
    _test = np.concatenate([y_test, X_test], axis=1)

    return _train, _val, _test

### Prepare training and inference scripts

By using the Scikit Learn estimator via the Sagemaker SDK, we can host and train models on Amazon Sagemaker.

For training, we do the following:

1. Prepare a training script - this script will execute the training logic within a SageMaker managed Scikit Learn container.


2. Create a `sagemaker.sklearn.estimator.SKLearn` estimator


3. Call the estimators `.fit()` method.

For more information on using scikit learn with the Sagemaker SDK, see the docs [here.](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/using_sklearn.html)

Below, we will create the training script called `training.py` that will be located at the root of a dicrectory called `source_dir`.

In this example, we will be training a [RandomForestRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) model that will later be used for inference in predicting house prices.

**NOTE:** You would modify the script below to implement your own training logic.

In [9]:
!mkdir $SOURCE_DIR

In [10]:
%%writefile $SOURCE_DIR/$TRAINING_FILE

import argparse
import os

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import joblib

if __name__ == "__main__":
    print("extracting arguments")
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    # to simplify the demo we don't use all sklearn RandomForest hyperparameters
    parser.add_argument("--n-estimators", type=int, default=10)
    parser.add_argument("--min-samples-leaf", type=int, default=3)

    # Data, model, and output directories
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--validation", type=str, default=os.environ.get("SM_CHANNEL_VALIDATION"))
    parser.add_argument("--model-name", type=str)

    args, _ = parser.parse_known_args()

    print("reading data")
    print("model_name: {}".format(args.model_name))

    train_file = os.path.join(args.train, args.model_name + "_train.csv")
    train_df = pd.read_csv(train_file)  # read in the training data

    val_file = os.path.join(args.validation, args.model_name + "_val.csv")
    test_df = pd.read_csv(os.path.join(val_file))  # read in the test data

    # Matrix representation of the data
    print("building training and testing datasets")
    X_train = train_df[train_df.columns[1 : train_df.shape[1]]]
    X_test = test_df[test_df.columns[1 : test_df.shape[1]]]
    y_train = train_df[train_df.columns[0]]
    y_test = test_df[test_df.columns[0]]

    # fitting the model
    print("training model")
    model = RandomForestRegressor(
        n_estimators=args.n_estimators, min_samples_leaf=args.min_samples_leaf, n_jobs=-1
    )

    model.fit(X_train, y_train)

    # print abs error
    print("validating model")
    abs_err = np.abs(model.predict(X_test) - y_test)

    # print couple perf metrics
    for q in [10, 50, 90]:
        print("AE-at-" + str(q) + "th-percentile: " + str(np.percentile(a=abs_err, q=q)))

    # persist model
    path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model, path)
    print("model persisted at " + path)

Writing source_dir/training.py


When using multi-model endpoints with the Sagemaker managed Scikit Learn container, we need to provide an entry point script for
inference that will **at least** load the saved model.

We will now create this script and call it `inference.py` and store it at the root of a directory called `source_dir`. This is the same directory which contains our `training.py` script.

**Note:** You could place the below `model_fn` function within the `training.py` script (above the main guard) if you prefer to have a single script.

**Note:** You would modify the script below to implement your own inferencing logic.

Additional information on model loading and model serving for Scikit Learn on SageMaker can be found [here.](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/using_sklearn.html#deploy-a-scikit-learn-model)

In [11]:
%%writefile $SOURCE_DIR/$INFERENCE_FILE

import os
import joblib


def model_fn(model_dir):
    print("loading model.joblib from: {}".format(model_dir))
    loaded_model = joblib.load(os.path.join(model_dir, "model.joblib"))
    return loaded_model

Writing source_dir/inference.py


### Kick off a model training job for each housing location

In [12]:
def save_data_locally(location, train, val, test, columns):
    _header = ",".join(columns)

    os.makedirs(f"data/{location}/train")
    np.savetxt(
        f"data/{location}/train/{location}_train.csv",
        train,
        delimiter=",",
        fmt="%.2f",
        header=_header,
    )

    os.makedirs(f"data/{location}/val")
    np.savetxt(
        f"data/{location}/val/{location}_val.csv", val, delimiter=",", fmt="%.2f", header=_header
    )

    os.makedirs(f"data/{location}/test")
    np.savetxt(
        f"data/{location}/test/{location}_test.csv", test, delimiter=",", fmt="%.2f", header=_header
    )

In [13]:
import shutil
import os

estimators = []

_houses = gen_houses(NUM_HOUSES)
_train, _val, _test = split_data(_houses)
save_data_locally(LOCATION, _train, _val, _test, _houses.columns)

### Launch a single training job for a given housing location
There is nothing specific to multi-model endpoints in terms of the models it will host. They are trained in the same way as all other SageMaker models. Here we are using the Scikit Learn estimator and not waiting for the job to complete.

In [14]:
from sagemaker.sklearn.estimator import SKLearn


# clear out old versions of the data
s3_bucket = s3.Bucket(BUCKET)
full_input_prefix = f"{DATA_PREFIX}/model_prep/{LOCATION}"
s3_bucket.objects.filter(Prefix=full_input_prefix + "/").delete()

# upload the entire set of data for all three channels
local_folder = f"data/{LOCATION}"
inputs = sagemaker_session.upload_data(path=local_folder, key_prefix=full_input_prefix)

print(f"Training data uploaded: {inputs}")

_job = "skl-{}".format(LOCATION.replace("_", "-"))
full_output_prefix = f"{DATA_PREFIX}/model_artifacts/{LOCATION}"
s3_output_path = f"s3://{BUCKET}/{full_output_prefix}"

code_location = f"s3://{BUCKET}/{full_input_prefix}/code"

# Add code_location argument in order to ensure that code_artifacts are stored in the same place.
estimator = SKLearn(
    entry_point=TRAINING_FILE,  # script to use for training job
    role=role,
    source_dir=SOURCE_DIR,  # Location of scripts
    instance_count=1,
    instance_type=TRAIN_INSTANCE_TYPE,
    framework_version="1.2-1",  # 1.2-1 is the latest version
    output_path=s3_output_path,  # Where to store model artifacts
    base_job_name=_job,
    code_location=code_location,  # This is where the .tar.gz of the source_dir will be stored
    metric_definitions=[{"Name": "median-AE", "Regex": "AE-at-50th-percentile: ([0-9.]+).*$"}],
    hyperparameters={"n-estimators": 100, "min-samples-leaf": 3, "model-name": LOCATION},
)

DISTRIBUTION_MODE = "FullyReplicated"

train_input = TrainingInput(
    s3_data=inputs + "/train", distribution=DISTRIBUTION_MODE, content_type="csv"
)

val_input = TrainingInput(
    s3_data=inputs + "/val", distribution=DISTRIBUTION_MODE, content_type="csv"
)

remote_inputs = {"train": train_input, "validation": val_input}

Training data uploaded: s3://sagemaker-us-west-2-258565740036/DEMO_SCRIPT_MODE_SCIKIT_V1/model_prep/NewYork_NY


## Wait for the job to finish

In [15]:
estimator.fit(remote_inputs, wait=True)

INFO:sagemaker:Creating training-job with name: skl-NewYork-NY-2025-07-22-08-21-30-835


2025-07-22 08:21:31 Starting - Starting the training job...
2025-07-22 08:21:51 Starting - Preparing the instances for training...
2025-07-22 08:22:29 Downloading - Downloading the training image......
2025-07-22 08:23:35 Training - Training image download completed. Training in progress.
2025-07-22 08:23:35 Uploading - Uploading generated training model.2025-07-22 08:23:31,029 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2025-07-22 08:23:31,032 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-07-22 08:23:31,035 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2025-07-22 08:23:31,049 sagemaker_sklearn_container.training INFO     Invoking user training script.
2025-07-22 08:23:31,246 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-07-22 08:23:31,249 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons ins

## Model data

You can check where sagemaker has created the model artifact archive folder.

In [16]:
estimator.model_data

's3://sagemaker-us-west-2-258565740036/DEMO_SCRIPT_MODE_SCIKIT_V1/model_artifacts/NewYork_NY/skl-NewYork-NY-2025-07-22-08-21-30-835/output/model.tar.gz'

## Create the multi-model endpoint with the SageMaker SDK

### Create a SageMaker Model from the `estimator`

From an `estimator` object, you can create a "deployable" model, where you can also specify the `entry_point`, i.e., the python script that can handle the incoming request. To customize your inference on SageMaker hosting, you can build out an `inference.py` file that initializes your model, defines the input and output structure, and produces your inference results. In this file, you can define four functions:

- `model_fn` – Initializes your model
- `input_fn` – Defines how your data should be input and how to convert to a usable format
- `predict_fn` – Takes the input data and receives the prediction
- `output_fn` – Converts the prediction into an API call format

In out example, we only provide the custom `model_fn` that overwrites the default method to tell SageMaker how to load the model in memory when the worker is initialized. All other functions uses the default methods.

In [17]:
# inference.py is the entry_point for when we deploy the model
# Note how we do NOT specify source_dir again, this information is inherited from the estimator
model = estimator.create_model(role=role, entry_point="inference.py")

## Deploy the Model Endpoint

You need to consider the appropriate instance type and number of instances for the projected prediction workload the model you plan to host behind your endpoint.

In [18]:
predictor = model.deploy(
    initial_instance_count=1, instance_type=ENDPOINT_INSTANCE_TYPE, endpoint_name=ENDPOINT_NAME
)

INFO:sagemaker:Creating model with name: skl-NewYork-NY-2025-07-22-08-26-05-067
INFO:sagemaker:Creating endpoint-config with name script-mode-sklearn-housing-V1-2025-07-22-08-21-19
INFO:sagemaker:Creating endpoint with name script-mode-sklearn-housing-V1-2025-07-22-08-21-19


------!

## Get predictions from the endpoint

Recall that ```model.deploy()``` returns a [RealTimePredictor](https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/predictor.py#L35) that we saved in a variable called ```predictor```.

We will use ```predictor``` to submit requests to the endpoint.

### Invoking models on a multi-model endpoint
Notice the higher latencies on the first invocation of any given model. This is due to the time it takes SageMaker to download the model to the Endpoint instance and then load the model into the inference container. Subsequent invocations of the same model take advantage of the model already being loaded into the inference container.

In [23]:
start_time = time.time()
payload = gen_random_house()[1:]
predicted_value = predictor.predict(data=[payload])

duration = time.time() - start_time
print(payload)
print("${:,.2f}, took {:,d} ms\n".format(predicted_value[0], int(duration * 1000)))

[1979, 2858, 4, 2.0, 0.92, 1]
$338,707.16, took 50 ms



## Using Boto APIs to invoke the endpoint

While developing interactively within a Jupyter notebook, since `.deploy()` returns a `RealTimePredictor` it is a more seamless experience to start invoking your endpoint using the SageMaker SDK. You have more fine grained control over the serialization and deserialization protocols to shape your request and response payloads to/from the endpoint. This is the approach we demonstrated above where the `RealTimePredictor` was stored in the variable `predictor`.

This is great for iterative experimentation within a notebook. Furthermore, should you have an application that has access to the SageMaker SDK, you can always import `RealTimePredictor` and attach it to an existing endpoint - this allows you to stick to using the high level SDK if preferable.

Additional documentation on `RealTimePredictor` can be found [here.](https://sagemaker.readthedocs.io/en/stable/api/inference/predictors.html?highlight=RealTimePredictor#sagemaker.predictor.RealTimePredictor)

The lower level Boto3 SDK may be preferable if you are attempting to invoke the endpoint as a part of a broader architecture.

Imagine an API gateway frontend that uses a Lambda Proxy in order to transform request payloads before hitting a SageMaker Endpoint - in this example, Lambda does not have access to the SageMaker Python SDK, and as such, Boto3 can still allow you to interact with your endpoint and serve inference requests.

Boto3 allows for quick injection of ML intelligence via SageMaker Endpoints into existing applications with minimal/no refactoring to existing code.

Boto3 will submit your requests as a binary payload, while still allowing you to supply your desired `Content-Type` and `Accept` headers with serialization being handled by the inference container in the SageMaker Endpoint.

Additional documentation on `.invoke_endpoint()` can be found [here.](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker-runtime.html)

In [24]:
import boto3
import json

runtime_sm_client = boto3.client(service_name="sagemaker-runtime")


def predict_one_house_value(features):
    print(f"predict price of this house: {features}")

    float_features = [float(i) for i in features]

    body = json.dumps([float_features])
    start_time = time.time()

    response = runtime_sm_client.invoke_endpoint(
        EndpointName=ENDPOINT_NAME, ContentType="application/json", Body=body
    )

    predicted_value = json.loads(response["Body"].read())[0]

    duration = time.time() - start_time

    print("${:,.2f}, took {:,d} ms\n".format(predicted_value, int(duration * 1000)))

In [25]:
predict_one_house_value(gen_random_house()[1:])

predict price of this house: [2002, 2854, 4, 3.0, 0.97, 3]
$452,180.45, took 99 ms



## Clean up
Here, to be sure we are not billed for endpoints we are no longer using, we clean up.

In [27]:
predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: script-mode-sklearn-housing-V1-2025-07-22-08-21-19
INFO:sagemaker:Deleting endpoint with name: script-mode-sklearn-housing-V1-2025-07-22-08-21-19


Remove folders created

In [26]:
!rm -rf data source_dir

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.

![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-1/advanced_functionality|multi_model_sklearn_home_value|sklearn_multi_model_endpoint_home_value.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-2/advanced_functionality|multi_model_sklearn_home_value|sklearn_multi_model_endpoint_home_value.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-1/advanced_functionality|multi_model_sklearn_home_value|sklearn_multi_model_endpoint_home_value.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ca-central-1/advanced_functionality|multi_model_sklearn_home_value|sklearn_multi_model_endpoint_home_value.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/sa-east-1/advanced_functionality|multi_model_sklearn_home_value|sklearn_multi_model_endpoint_home_value.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-1/advanced_functionality|multi_model_sklearn_home_value|sklearn_multi_model_endpoint_home_value.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-2/advanced_functionality|multi_model_sklearn_home_value|sklearn_multi_model_endpoint_home_value.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-3/advanced_functionality|multi_model_sklearn_home_value|sklearn_multi_model_endpoint_home_value.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-central-1/advanced_functionality|multi_model_sklearn_home_value|sklearn_multi_model_endpoint_home_value.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-north-1/advanced_functionality|multi_model_sklearn_home_value|sklearn_multi_model_endpoint_home_value.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-1/advanced_functionality|multi_model_sklearn_home_value|sklearn_multi_model_endpoint_home_value.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-2/advanced_functionality|multi_model_sklearn_home_value|sklearn_multi_model_endpoint_home_value.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-1/advanced_functionality|multi_model_sklearn_home_value|sklearn_multi_model_endpoint_home_value.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-2/advanced_functionality|multi_model_sklearn_home_value|sklearn_multi_model_endpoint_home_value.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-south-1/advanced_functionality|multi_model_sklearn_home_value|sklearn_multi_model_endpoint_home_value.ipynb)
